In [37]:
import numpy as np
import random
import time

In [92]:
class Cluster():
    def __init__(self, cluster_id, L, cell):
        self.cluster_id = cluster_id
        self.cell_pattern = 0b0
        self.lattice_dimension = L
        self.cells = [cell]
        self.set_bits(cell)
        
    def add_cluster(self, other):
        self.cells += other.cells
        for cell in other.cells:
            maskx = 1 << cell.x
            masky = 1 << (cell.y + self.lattice_dimension)
            self.cell_pattern = self.cell_pattern | maskx
            self.cell_pattern = self.cell_pattern | masky
        print(bin(self.cell_pattern))
    
    def set_bits(self, cell):
        maskx = 1 << cell.x
        masky = 1 << (cell.y + self.lattice_dimension)
        self.cell_pattern = self.cell_pattern | maskx
        self.cell_pattern = self.cell_pattern | masky
    
    def get_cluster_size(self):
        return len(self.cells)
    
    def does_percolate(self):
        horizontal = False
        vertical = False
        
        testx = 2**self.lattice_dimension - 1
        testy = 2**(2 * self.lattice_dimension) - testx - 1
       
        horizontal = ((self.cell_pattern & testx) == testx)
        vertical = ((self.cell_pattern & testy) == testy)
        
        return(horizontal or vertical)
    

In [93]:
bin(2^4)

'0b110'

In [94]:
class Cell():
    def __init__(self, cell_id, x, y):
        self.cell_id = cell_id
        self.x = x
        self.y = y
        self.is_occupied = False
        self.next_cell = None
        self.root = self
 

    def get_root(self):
        if self.root == self:
            return self
        else:
            root = self.root
            while root != root.get_root():
                root = root.get_root()
            else:
                return root
   
    def set_root(self, root):
        self.root = root
    
    def set_next(self, other):
        self.next_cell = other
    
    def occupy_cell(self):
        self.is_occupied = True
    
    def compress_path(self, root):
        # redirect cell's root to new root
        if self.get_root() != self:
            self.get_root().set_root(root)
        
        # now set original cell's root
        self.set_root(root)
        # now set root of all cells that are linked to original cell
        cell = self
        while cell.next_cell:
            if cell.next_cell.get_root() == root:
                break
            cell.next_cell.set_root(root)
            cell = cell.next_cell
        else:
            pass
            cell.set_root(root)
            
            
    def get_distance_to_next(self):
        if self.next_cell:
            dx = self.next_cell.x - self.x
            dy = self.next_cell.y - self.y
            #check for wrapping
            if np.abs(dx) > 1:
                dx = -1 * np.sign(dx) 
            if np.abs(dy) > 1:
                dy = -1 * np.sign(dy) 
        else:
            dx = 0
            dy = 0
            
        return (dx, dy)
    
    def get_spanning_vector(self):
        cell = self
        dx = 0
        dy = 0
        while cell.next_cell and cell != self.root:
            (x, y) = cell.get_distance_to_next()
            dx += x
            dy += y
            cell = cell.next_cell
        return (dx, dy)
    
    
    def get_neighbours(self, L):
        xl = self.x - 1 if self.x > 0 else (L - 1)
        yu = self.y + 1 if self.y < (L - 1) else 0
        xr = self.x + 1 if self.x < (L - 1) else 0
        yd = self.y - 1 if self.y > 0 else (L - 1)
        return [(xl, y), (x, yu), (xr, y), (x, yd)]
    
    
    def __eq__(self, other):
        return (self.cell_id == other.cell_id)    
    
    
    def __ne__(self, other):
        return (self.cell_id != other.cell_id)    
    
    def is_equal(self, other):
        return (self.cell_id != other.cell_id) 
    
    

In [95]:
# some helper functions

def plot_lattice(lattice, N):
    tmp = np.full(lattice.shape, 0, dtype='i')
    for i in range(N):
        x = int(i % L)
        y = int((i - x) / L)
        if lattice[x][y].is_occupied:
            tmp[x][y] = lattice[x][y].get_root().cell_id
    print(tmp)

def print_cluster(clustersize):
    for cluster, size in clustersize.items():
        if size > 0:
            print(cluster, " ", size)

In [105]:
L = 10
N = L * L

# create a lattice and fill it with 'empty' cells
lattice = np.ndarray([L, L], dtype = type(Cell))
coordinates = []
for i in range(N):
    x = int(i % L)
    y = int((i - x) / L)
    lattice[x][y] = Cell(i + 1, x, y)
    coordinates.append((x, y))
    
    
# shuffle coordinates to randomly select cells to be filled:
np.random.shuffle(coordinates)

In [106]:
start = time.time()

# keep track of cluster sizes
sizes = [0 for i in range(N + 1)]
ids = [i for i in range(N + 1)]
cluster_size = dict(zip(ids, sizes))

clusters = {}

for (x, y) in coordinates:
        
    lattice[x][y].occupy_cell() 
    #print(L * y + x + 1, x,y, 'VVVVVVV')
    plot_lattice(lattice, N)
    print()
    cluster_size[lattice[x][y].cell_id] = 1
    
    clusters[lattice[x][y].cell_id] = Cluster(lattice[x][y].cell_id, L, lattice[x][y])
    
    #check for neighbouring cells:
    nn_coordinates = lattice[x][y].get_neighbours(L)
    
    tmp = lattice.copy()
    
    for (nx, ny) in nn_coordinates:
        if lattice[nx][ny].is_occupied:
           
            r1 = lattice[x][y].get_root()
            id1 = lattice[x][y].cell_id
            size1 = cluster_size[r1.cell_id]
            
            r2 = lattice[nx][ny].get_root()
            id2 = lattice[nx][ny].cell_id
            size2 = cluster_size[r2.cell_id]
    
            if r1 != r2:
                if size1 < size2:
                    lattice[x][y].set_next(lattice[nx][ny])
                    lattice[x][y].compress_path(r2)
                    cluster_size[r1.cell_id] = 0
                    cluster_size[r2.cell_id] += size1
                    clusters[r2.cell_id].add_cluster(clusters[r1.cell_id])
                    clusters.pop(r1.cell_id)
                    if clusters[r2.cell_id].does_percolate():
                        print(bin(clusters[r2.cell_id].cell_pattern))
                        plot_lattice(lattice, N)
                else: 
                    lattice[nx][ny].set_next(lattice[x][y])
                    lattice[nx][ny].compress_path(r1)
                    cluster_size[r1.cell_id] += size2
                    cluster_size[r2.cell_id] = 0 
                    clusters[r1.cell_id].add_cluster(clusters[r2.cell_id])
                    clusters.pop(r2.cell_id)
                    if clusters[r1.cell_id].does_percolate():
                        print(bin(clusters[r1.cell_id].cell_pattern))
                        plot_lattice(lattice, N)
            else:
                lattice[x][y].set_next(lattice[nx][ny])
              #  print()
              #  plot_lattice(lattice, N)
                print(lattice[x][y].cell_id, lattice[x][y].get_root().cell_id, lattice[x][y].get_spanning_vector(), " - ", 
                      lattice[nx][ny].cell_id, lattice[nx][ny].get_root().cell_id, lattice[nx][ny].get_spanning_vector())
                (ax, ay) = lattice[x][y].get_spanning_vector()
                (bx, by) = lattice[nx][ny].get_spanning_vector()
                if np.abs(ax - bx + ay - by) > 2:
                    print("PERCOLATION", lattice[x][y].get_root().cell_id, cluster_size[lattice[x][y].get_root().cell_id])
                    plot_lattice(tmp, N)
                    print()
                    plot_lattice(lattice, N)
    #plot_lattice(lattice, N)
    #print(x,y, '^^^^^^^^')
stop = time.time()
print(stop - start)

[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 67  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]

[[ 0  0 21  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 67  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]

[[ 0  0 21  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 57 67  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]

0b11